In [21]:
# %matplotlib notebook
import pandas as pd
import numpy as np
import multiprocessing
import argparse
from datetime import timedelta
import pyvips
from PIL import Image
import time
import random
import gc
import logging
import os
from pathlib import Path
import sys
import matplotlib.pyplot as plt

sys.path.append('..')
from src.wsi import slides, filters, tiles, util
from src import dataset

BASE_PATH = Path(os.path.dirname(os.getcwd()))
DATA_PATH = BASE_PATH / 'data' / 'originals'
PROCESSED_DATA = BASE_PATH / 'data' / 'processed'

In [2]:
train = pd.read_csv(BASE_PATH / 'data' / 'testlow.csv')
row = train.iloc[2]
image = pyvips.Image.new_from_file(f'../data/originals/{row["filename"]}', access='sequential')
image.filename, image.width, image.height

('../data/originals/bu5xt1xm.tif', 157949, 72896)

In [3]:
image.get('bands'), image.get('coding'), image.get('format'), image.get('interpretation')

(3, 'none', 'uchar', 'srgb')

In [8]:
image.tiffsave('test.tif', compression="deflate",
                 tile=True, tile_width=256, tile_height=256,
                 pyramid=True)

In [56]:
def rimages(index, train, page):
    filename, res = train.loc[index, ['filename', 'resolution']]
    resizing_factor = res / 0.25
    image = pyvips.Image.new_from_file(f'../data/originals/{filename}', page=page)
    width, height = image.width, image.height
    image = image.affine((resizing_factor, 0, 0, resizing_factor))
    return width, height, image, image.width, image.height

In [3]:
images = pyvips.Image.new_from_file('../data/originals/qmsqv731.tif', page=8)
images.height, images.width
img = images.write_to_file('test.jpg')
# img = np.ndarray(buffer=images.write_to_memory(), dtype=np.uint8, shape=[images.height, images.width, images.bands])
plt.imshow(plt.imread('test.jpg'))


In [10]:
plt.imshow(images.numpy() / 255.)
plt.show()

In [3]:
img = Image.open('test.jpg')
plt.imshow(img)

In [6]:
dftrain = dataset.get_dftrain('../workspace/tiles/data/processed/48/1/')
print(dftrain.n_tiles.value_counts())
dftrain

training dataset: 1257 samples
training dataset: 30114 tiles
n_tiles
24    30024
22       22
20       20
16       16
13       13
12       12
7         7
Name: count, dtype: int64


,tissue_id,filename,relapse,split,n_tiles,path,tile_id,is_valid
0,00shcdng,00shcdng.tif,0,1,24,../workspace/tiles/data/processed/48/1/00shcdng_0.jpeg,0,0
1,00shcdng,00shcdng.tif,0,1,24,../workspace/tiles/data/processed/48/1/00shcdng_1.jpeg,1,0
2,00shcdng,00shcdng.tif,0,1,24,../workspace/tiles/data/processed/48/1/00shcdng_2.jpeg,2,0
3,00shcdng,00shcdng.tif,0,1,24,../workspace/tiles/data/processed/48/1/00shcdng_3.jpeg,3,0
4,00shcdng,00shcdng.tif,0,1,24,../workspace/tiles/data/processed/48/1/00shcdng_4.jpeg,4,0
...,...,...,...,...,...,...,...,...
30109,zyo5fbtd,zyo5fbtd.tif,1,0,24,../workspace/tiles/data/processed/48/1/zyo5fbtd_19.jpeg,19,1
30110,zyo5fbtd,zyo5fbtd.tif,1,0,24,../workspace/tiles/data/processed/48/1/zyo5fbtd_20.jpeg,20,1
30111,zyo5fbtd,zyo5fbtd.tif,1,0,24,../workspace/tiles/data/processed/48/1/zyo5fbtd_21.jpeg,21,1
30112,zyo5fbtd,zyo5fbtd.tif,1,0,24,../workspace/tiles/data/processed/48/1/zyo5fbtd_22.jpeg,22,1


In [45]:
dls = dataset.get_dataloader(dftrain)
#tr, val = dblock.splitter(dblock.get_items(dftrain))
#dataset.show_tile_batch(dls)

In [46]:
tr = dls.train

In [4]:
%matplotlib inline

In [44]:
x, y = dls.valid.one_batch()
x[0].shape

torch.Size([16, 3, 768, 768])